In [1]:
import preamble_test
import symbol_mod_QAM
import symbol_demod_QAM
import numpy as np
import preamble_generator
import data_source
import mode_preconfiguration
import pulse_shaping
import matched_filtering
import time
import matplotlib.pyplot as plt

def ModemTest(scheme, preamble_ones_length, M, R, L, packet_length_multiplier, pulse_shape):
    # FOR INCORPORATING PULSE SHAPING

    fs = 750000.0	       # maximum output sampling rate sustainable for pluto streaming: 2750000 (Hz), for m2k streaming: 750000 (Hz) 
    Ts = 1.0 / fs          # sampling period in seconds
    f0 = 0.0               # homodyne (0 HZ IF)
    M = M                  # oversampling factor
    T = M*Ts               # symbol period in seconds
    Rs = 1/T               # symbol rate
    segment_size = 1504    # One Transport Stream (TS) packet=188Bytes=1504 bits
    R = R                 # Packet Ratio: number of segments contained in our larger OOK packet 
    N = R*segment_size     # OOK Packet Length (equals R* segment_size)
    L = L                  # pulse shaping filter length
    pulse_shape = pulse_shape # type of pulse shaping
    scheme = scheme        # Modulation scheme
    alpha = 0.9 #roll-off factor of the RRC matched-filter
    
    Mode = 4
    b = "40k"             #bandwidth of Iperf client (UDP test)
    test_packet_num = 20  #number of ADALM packets transmitted in Iperf test
    
    #############################################
    # Generate Packet
    
    segment_size = segment_size*packet_length_multiplier
    
    serverSock, generated_sequence, sequence_counter, l = mode_preconfiguration.tx_mode_preconfig(Mode, R, segment_size, N, b, test_packet_num)

    preamble = preamble_generator.preamble_generator(preamble_ones_length)
    #preamble = np.zeros(200)

    Bits = data_source.data_source(Mode, serverSock, generated_sequence, sequence_counter, l)

    packet_bits = np.append(preamble, Bits)

    preamble_length = len(preamble)

    total_samples, samples_perbit, fs_in, Ts_in =  mode_preconfiguration.rx_mode_preconfig(scheme,N,preamble_length,M,fs)

    ################################
    # Generate Preamble check for matched filter
    known_preamble_bits = preamble_generator.preamble_generator(preamble_ones_length)
    known_preamble_symbols = symbol_mod_QAM.symbol_mod(known_preamble_bits, "OOK", len(known_preamble_bits))
    known_preamble = np.abs(pulse_shaping.pulse_shaping(known_preamble_symbols, samples_perbit, fs_in, 'rect', 0.9, 8))

    ######################
    # Generate symbols
    start = time.time()
    #print("1")
    baseband_symbols = symbol_mod_QAM.symbol_mod(packet_bits, scheme, preamble_length)

    ######################
    # Pulse Shaping
    baseband = pulse_shaping.pulse_shaping(baseband_symbols, M, fs, pulse_shape, 0.9, 8)

    bb_amplitude = 1
    buffer = bb_amplitude*baseband

    buffer_IQ = buffer
    
    ######################
    # Separate Data Streams
    I_data = np.real(buffer_IQ)
    Q_data = np.imag(buffer_IQ)

    a_in1 = buffer

    #############################
    # Simulate "Phase Sync"
    
    oldN = N
    if(scheme == "QPSK"):
            N = int(N/2)

    if(scheme == "QAM"):
            N = int(N/4)

    payload_start = len(known_preamble)

    payload_before_correction = a_in1[payload_start:(payload_start + N*samples_perbit)]

    ones_length = preamble_length - 20

    payload_and_ones = a_in1[int(payload_start-ones_length*samples_perbit):(payload_start + N*samples_perbit)]

    ### In rf tx/rx scripts, phase sync goes here, but no need for phase sync in simulation
    
    payload_corrected = payload_and_ones[ones_length*samples_perbit:]

    baseband_signal_I_new = np.real(payload_corrected)
    baseband_signal_Q_new = np.imag(payload_corrected)

    N = oldN
    
    ######################################
    # Matched filtering

    symbols_I = matched_filtering.matched_filtering(baseband_signal_I_new, samples_perbit, fs_in, alpha, L, pulse_shape)

    symbols_Q = matched_filtering.matched_filtering(baseband_signal_Q_new, samples_perbit, fs_in, alpha, L, pulse_shape)

    #Remove the samples at the beginning and samples at the end caused by group delay of the filter

    if (pulse_shape == 'rect'):
        symbols_I = symbols_I[1:]
        symbols_Q = symbols_Q[1:]
    elif (pulse_shape == 'rrc'):
        symbols_I = symbols_I[L-1:(len(symbols_I))-L]
        symbols_Q = symbols_Q[L-1:(len(symbols_Q))-L]
        
    symbols = np.array([symbols_I,symbols_Q])
        
    ######################################
    # Calculate Channel Gain

    if (scheme == 'QPSK'):  
        channel_gain = np.mean(np.abs(symbols_I))/(2/3)
    elif (scheme == 'QAM'):  
        channel_gain = np.mean(np.abs(symbols_I))/(2)
    else:
        channel_gain = np.mean(np.abs(symbols_I))

    ######################################
    # Demodulation

    a_demodulated = symbol_demod_QAM.symbol_demod(symbols, scheme, channel_gain, preamble_length)

    #print("Demodulated bits match generated bits =")
    if (pulse_shape == 'rrc'):
        if(np.array_equal(Bits[28:-32], a_demodulated, equal_nan=False)):
            print("BER:" + str(0.0))
            #BER = 0
        else:
            print("Non-zero BER")
            #BER = "non-zero"
    else:
        if (np.array_equal(packet_bits[200:], a_demodulated, equal_nan=False)):
            print("BER:" + str(0.0))
            #BER = 0
        else:
            print("Non-zero BER")
            #BER = "non-zero"
    

    ######################################
    # Throughput calculation
    
    packet_kbits = len(Bits)
    total_kbits = 0
    total_kbits = total_kbits+packet_kbits

    throughput = total_kbits/(time.time()-start)
    #print("2")
    
    #throughput = total_kbits
    
    return throughput

def ConstellationPlot(symbols_I, symbols_Q, channel_gain):
    showplot = True
    if(showplot==True):
            #plt.ylim((-2, 1))
            #plt.xlim((1, 2))
            for i in range(len(symbols_I)):
                    plt.plot(symbols_I[i]/channel_gain,symbols_Q[i]/channel_gain, color='blue', marker='o', markersize=1)
            plt.show()
            
if __name__ == "__main__":
    
        iterations = 20
        
    ### OPTIMIZED SYSTEM
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("OOK", 180, 4, 5, 8, 1, 'rect')
            #print(throughput) 
            total_throughput = total_throughput + throughput
            
        #Average_throughput_M4 = (total_throughput-60)/(time.time()-start)
        Average_throughput_Optimized = total_throughput/50
        
        print("Optimized Throughput:")
        print(Average_throughput_Optimized)
    
    ### Upsampling factor (QAM)
        Marray = []
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 4, 5, 8, 1, 'rrc')
            #print(throughput) 
            total_throughput = total_throughput + throughput
            
        #Average_throughput_M4 = (total_throughput-60)/(time.time()-start)
        Average_throughput_M4 = total_throughput/iterations
        print(Average_throughput_M4)
        Marray.append(Average_throughput_M4)
        
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_M6 = total_throughput/iterations
        #print(Average_throughput_M6)
        Marray.append(Average_throughput_M6)
        
        start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 8, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_M8 = total_throughput/iterations
        #print(Average_throughput_M8)
        Marray.append(Average_throughput_M8)
        
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 12, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_M12 = total_throughput/iterations
        #print(Average_throughput_M12)
        Marray.append(Average_throughput_M12)
        
        start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 24, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_M24 = total_throughput/iterations
        #print(Average_throughput_M24)
        Marray.append(Average_throughput_M24)
        
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 36, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_M36 = total_throughput/iterations
        #print(Average_throughput_M36)
        Marray.append(Average_throughput_M36)
        
        plt.plot([4, 6, 8, 12, 24, 36], Marray, 'ro')
        plt.xlabel('Upsampling Factor')
        plt.ylabel('Throughput (bits/sec)')
        plt.title('Upscaling Factor vs Throughput for QAM')
        plt.show()
            
            # plot throughput vs upsampling factor
    
    ### Averaging throughput for QAM
        
        modnames = ["QAM", "QPSK", "OOK"]
        modarray = []
        start = time.time()
        total_throughput = 0
        
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_QAM = total_throughput/iterations
        modarray.append(Average_throughput_QAM)
    
    ### Averaging throughput for QPSK
        #start = time.time()
        total_throughput = 0
        
        for i in range(iterations):
            throughput = ModemTest("QPSK", 180, 6, 5, 8, 1, 'rect')
            total_throughput = total_throughput + throughput
            
        Average_throughput_QPSK = total_throughput/iterations
        modarray.append(Average_throughput_QPSK)
    
    ### Averaging throughput for OOK
        #start = time.time()
        total_throughput = 0
        
        for i in range(iterations):
            throughput = ModemTest("OOK", 180, 6, 5, 8, 1, 'rect')
            total_throughput = total_throughput + throughput
            
        Average_throughput_OOK = total_throughput/iterations
        modarray.append(Average_throughput_OOK)
        
        # Generate bar plot
        plt.bar(modnames, modarray)
        plt.xlabel('Modulation Scheme')
        plt.ylabel('Throughput (bits/sec)')
        plt.title('Modulation Scheme vs Throughput')
        plt.show()
    
    
    ### Preamble length
        prarray = []
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 10, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P10 = total_throughput/50
        prarray.append(Average_throughput_P10)
            
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 100, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P100 = total_throughput/50
        prarray.append(Average_throughput_P100)
            
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 1000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P1000 = total_throughput/50
        prarray.append(Average_throughput_P1000)
        
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 5000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P5000 = total_throughput/50
        prarray.append(Average_throughput_P5000)
            
            
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 10000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P10000 = total_throughput/50
        prarray.append(Average_throughput_P10000)
        
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 20000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P20000 = total_throughput/50
        prarray.append(Average_throughput_P20000)
        
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 40000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P40000 = total_throughput/50
        prarray.append(Average_throughput_P40000)
        
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 60000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P60000 = total_throughput/50
        prarray.append(Average_throughput_P60000)
        
        #start = time.time()
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 80000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P80000 = total_throughput/50
        prarray.append(Average_throughput_P80000)
        
        total_throughput = 0
        for i in range(50):
            throughput = ModemTest("QAM", 100000, 6, 5, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_P100000 = total_throughput/50
        prarray.append(Average_throughput_P100000)
            
            # plot throughput vs preamble ones length
        plt.plot([10, 100, 1000, 5000, 10000, 20000, 40000, 60000, 80000, 100000], prarray, 'ro')
        plt.xlabel('Preamble Length (bits)')
        plt.ylabel('Throughput (bits/sec)')
        plt.title('Preamble Length vs Throughput for QAM')
        plt.show()
            
    ### Packet ratio
        parray = []
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 6, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R6 = total_throughput/iterations
        parray.append(Average_throughput_R6)
            
        start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 8, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R8 = total_throughput/iterations
        parray.append(Average_throughput_R8)
            
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 10, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R10 = total_throughput/iterations
        parray.append(Average_throughput_R10)
            
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 12, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R12 = total_throughput/iterations
        parray.append(Average_throughput_R12)
        
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 16, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R16 = total_throughput/iterations
        parray.append(Average_throughput_R12)
        
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 20, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R20 = total_throughput/iterations
        parray.append(Average_throughput_R12)
        
        #start = time.time()
        total_throughput = 0
        for i in range(iterations):
            throughput = ModemTest("QAM", 180, 6, 25, 8, 1, 'rrc')
            total_throughput = total_throughput + throughput
            
        Average_throughput_R20 = total_throughput/iterations
        parray.append(Average_throughput_R12)
            
            # plot throughput vs packet ratio
        plt.plot([6, 8, 10, 12, 16, 20, 25], parray, 'ro')
        plt.xlabel('Packet Ratio')
        plt.ylabel('Throughput (bits/sec)')
        plt.title('Packet Ratio vs Throughput for QAM')
        plt.show()
    
            
            


BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
BER:0.0
Optimized Throughput:
241742.6250700187
BER:0.0
BER:0.0
BER:0.0


KeyboardInterrupt: 